# Imports
All problem-speciffic files (code) in in the folder `MeshpLaplace2D`, namely the loading of pre-prepared mesh and definition of energy in jax.numpy.

In [1]:
from tools.minimizers import newton
from tools.sparse_solvers import HessSolverGenerator
from tools.jax_diff import EnergyDerivator as EnergyDerivator
from pLaplace2D.jax_energy import J
from pLaplace2D.mesh import MeshpLaplace2D

from jax import config

config.update("jax_enable_x64", True)

/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


# Setup of the problem

In [2]:
mesh = MeshpLaplace2D(mesh_level=7)
params, adjacency, u_init = mesh.get_data_jax()
energy_pLaplace = EnergyDerivator(J, params, adjacency, u_init)

F, dF, ddF = energy_pLaplace.get_derivatives()
ddf_with_solver = HessSolverGenerator(ddf=ddF, solver_type="amg", verbose=True, tol=1e-3)
print(energy_pLaplace.timings)

{'coloring': 0.482206375017995, 'compilation': 0.27728103697882034}


# Solution via Newton's method

In [3]:
res = newton(F, dF, ddf_with_solver, u_init, verbose=True, tolf=1e-5, linesearch_tol=1e-3)
res

Iterations in AMG solver: 4.
it=1, f=-0.00448, fstep = 5.21144e+05, ||g||=28062.38599, nitf=17, a=1.99965e+00
Iterations in AMG solver: 9.
it=2, f=-5.20909, fstep = 5.20461e+00, ||g||=0.22497, nitf=17, a=5.62308e-02
Iterations in AMG solver: 7.
it=3, f=-7.31352, fstep = 2.10443e+00, ||g||=5.44356, nitf=17, a=4.12779e-01
Iterations in AMG solver: 5.
it=4, f=-7.93687, fstep = 6.23352e-01, ||g||=3.79852, nitf=17, a=1.05137e+00
Iterations in AMG solver: 6.
it=5, f=-7.95820, fstep = 2.13328e-02, ||g||=0.54077, nitf=17, a=1.08356e+00
Iterations in AMG solver: 6.
it=6, f=-7.95829, fstep = 8.82177e-05, ||g||=0.03277, nitf=17, a=1.02074e+00


{'x': Array([-0.76453847, -0.65248595, -0.79502379, ..., -0.45099967,
        -0.45658825, -0.45024498], dtype=float64),
 'fun': array(-7.95829233),
 'nit': 6,
 'time': 5.870554598979652,
 'message': 'Stopping condition for g is satisfied'}